In [1]:
pd.set_option('display.max_rows', 500)

NameError: name 'pd' is not defined

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pdlt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.preprocessing import LabelEncoder

## Reading the data

In [3]:
train_data_df = pd.read_csv('./assets/train_users_2.csv/train_users_2.csv')

In [4]:
print train_data_df.shape
train_data_df.columns

(213451, 16)


Index([u'id', u'date_account_created', u'timestamp_first_active',
       u'date_first_booking', u'gender', u'age', u'signup_method',
       u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'country_destination'],
      dtype='object')

In [5]:
train_data_df['date_account_created'] = pd.to_datetime(train_data_df['date_account_created'])
train_data_df['year_acc_created'] = train_data_df.date_account_created.dt.year
train_data_df['month_acc_created'] = train_data_df.date_account_created.dt.month
train_data_df['day_acc_created'] = train_data_df.date_account_created.dt.day
train_data_df['dayofyear_acc_created'] = train_data_df.date_account_created.dt.dayofyear

In [6]:
_train_data_df = train_data_df.copy()#[train_data_df['year_acc_created'] == 2014]
# _2014_train_data_df = _2014_train_data_df[_2014_train_data_df['country_destination'] != 'NDF']
_train_data_df.shape

(213451, 20)

In [7]:
# train_data_df#.iloc[:,7:]#[train_data_df.columns[[(7:)]]]

*  ----------------------------------------------------------------- *
** In the sessions dataset, the data only dates back to 1/1/2014, while the users dataset dates back to 2010. **

In [8]:
sessions_df = pd.read_csv('./assets/sessions.csv/sessions.csv')

In [9]:
sessions_df.shape

(10567737, 6)

In [10]:
sessions_df.action = sessions_df.action.fillna('NAN')
sessions_df.action_type = sessions_df.action_type.fillna('NAN')
sessions_df.action_detail = sessions_df.action_detail.fillna('NAN')
sessions_df.device_type = sessions_df.device_type.fillna('NAN')

In [11]:
sessions_df.isnull().sum()

user_id           34496
action                0
action_type           0
action_detail         0
device_type           0
secs_elapsed     136031
dtype: int64

_____________________
## **Merging** 

In [12]:
sessions_df_temp = sessions_df.copy()
sessions_df_temp.set_index('user_id',drop=False, inplace=True)
users = _train_data_df['id']
sess_users = sessions_df['user_id'].unique()
print len(users)
print len(sess_users)

213451
135484


In [13]:
sessions_id = list(set(users).intersection(sess_users))
_sessions_df = sessions_df_temp.loc[sessions_id]
len(sessions_id)

73815

#### Examining Null values

In [14]:
# _2014_sessions_df.secs_elapsed.fillna('NAN', inplace=True)
print _sessions_df.shape
print _sessions_df.isnull().sum()
_sessions_df.head(1)

(5537957, 6)
user_id              0
action               0
action_type          0
action_detail        0
device_type          0
secs_elapsed     73815
dtype: int64


,user_id,action,action_type,action_detail,device_type,secs_elapsed
user_id,,,,,,
ey7zd06giy,ey7zd06giy,search_results,click,view_search_results,Mac Desktop,5770.0


In [16]:
s_temp = _sessions_df[_sessions_df.secs_elapsed.isnull()]
print s_temp.user_id.unique().size
print '\n', 'action unique values: ', s_temp.action.unique().size
print s_temp.action.value_counts().head(5)
print '\n', 'action type unique values: ', s_temp.action_type.unique().size
print s_temp.action_type.value_counts().head(5)
print '\n', 'action detail unique values: ', s_temp.action_detail.unique().size
print s_temp.action_detail.value_counts().head(5)

73815

action unique values:  63
create          41016
NAN             11363
authenticate    10980
active           5444
callback         2038
Name: action, dtype: int64

action type unique values:  9
submit              51516
message_post        11363
-unknown-            5928
partner_callback     2038
view                 1008
Name: action_type, dtype: int64

action detail unique values:  57
create_user     32337
message_post    11363
login           10439
signup           8570
-unknown-        5928
Name: action_detail, dtype: int64


It appears that each user in the dataset has one missing value in the secs elapsed column.

------------------
### Examining elemnts of each column

In [17]:
print (len(_sessions_df.action.unique()))
print (len(_sessions_df.action_type.unique()))
print (len(_sessions_df.action_detail.unique()))
print (len(_sessions_df.device_type.unique()))
# run corr

332
10
129
14


In [15]:
def percentile_col(col):
    percentile = []
    pt = 0
    for i in range(len(col)):
        value = col[i]
        s = col.sum()
        percent = round(100.0*value/s,3)
        pt += percent
        percentile.append(round(pt,3))
    return percentile

#### Action column

In [16]:
actions_gb_ = _sessions_df.groupby('action')['secs_elapsed'].agg([np.sum,np.mean,np.std,'count']).sort_values('sum',ascending=False)
actions_gb_['percentile'] = percentile_col(actions_gb_['sum'])
actions_gb = actions_gb_[actions_gb_.percentile < 90]
print len(actions_gb)
actions_gb.head()

36


,sum,mean,std,count,percentile
action,,,,,
show,2.096122e+10,16101.507637,86117.448392,1301817,18.753
index,1.386731e+10,29949.495131,107082.122210,463023,31.160
update,8.672156e+09,42077.213376,105926.230779,206101,38.919
search_results,8.004420e+09,19072.627886,65922.630950,419681,46.080
confirm_email,7.011423e+09,203253.231331,344920.690688,34496,52.353


#### Action details column

In [17]:
action_details_gb_ = _sessions_df.groupby('action_detail')['secs_elapsed'].agg([np.sum,np.mean,np.std,'count']).sort_values('sum',ascending=False)
action_details_gb_['percentile'] = percentile_col(action_details_gb_['sum'])
action_details_gb = action_details_gb_[action_details_gb_.percentile < 90]
print len(action_details_gb)
action_details_gb.head()

21


,sum,mean,std,count,percentile
action_detail,,,,,
view_search_results,2.039027e+10,22340.116143,74216.526999,912720,18.243
p3,1.535947e+10,25175.826417,106962.305495,610088,31.985
-unknown-,1.497179e+10,26789.151406,109450.176045,558875,45.380
confirm_email_link,7.011309e+09,204036.574106,345357.193465,34363,51.653
message_post,5.502601e+09,124481.964370,227046.425258,44204,56.576


#### Action type column

In [18]:
action_types_gb = _sessions_df.groupby('action_type')['secs_elapsed'].agg([np.sum,np.mean,np.std,'count']).sort_values('sum',ascending=False)
action_types_gb['percentile'] = percentile_col(action_types_gb['sum'])
action_types_gb = action_types_gb#[actions_type_gb_.percentile < 95]
print len(action_types_gb)
action_types_gb#.head()

10


,sum,mean,std,count,percentile
action_type,,,,,
view,4.182430e+10,25063.775145,104179.141921,1668715,37.419
click,2.252086e+10,21168.545085,91474.964107,1063883,57.568
-unknown-,1.497179e+10,26789.151406,109450.176045,558875,70.963
submit,1.402564e+10,45857.157016,128889.943337,305855,83.511
data,7.232136e+09,6087.057228,37501.457041,1188117,89.981
message_post,5.502601e+09,124481.964370,227046.425258,44204,94.904
NAN,4.851193e+09,7837.880310,66258.207860,618942,99.244
booking_request,7.050266e+08,66625.081837,164211.211725,10582,99.875
partner_callback,1.396080e+08,28107.103282,65561.431825,4967,100.000


#### Device column

In [19]:
device_type_gb_ = _sessions_df.groupby('device_type')['secs_elapsed'].agg([np.sum,np.mean,np.std,'count']).sort_values('sum',ascending=False)
device_type_gb_['percentile'] = percentile_col(device_type_gb_['sum'])
device_type_gb = device_type_gb_[device_type_gb_.percentile < 96]
print len(device_type_gb)
device_type_gb

6


,sum,mean,std,count,percentile
device_type,,,,,
Mac Desktop,4.297800e+10,20241.954126,90168.452303,2123214,38.451
Windows Desktop,3.501361e+10,22886.192443,98631.068275,1529901,69.777
iPhone,1.447585e+10,18602.955034,94215.619467,778148,82.728
iPad Tablet,8.119515e+09,21380.422578,89728.556061,379764,89.992
Android App Unknown Phone/Tablet,3.514827e+09,13290.682978,68551.947303,264458,93.137
Android Phone,2.905329e+09,13978.074862,81083.495896,207849,95.736


#### Filtering out rows

In [20]:
actions = actions_gb.index
action_types = action_types_gb.index
action_details = action_details_gb.index
device_types = device_type_gb.index

In [21]:
__sessions_df = _sessions_df[_sessions_df.action.isin(actions)]
__sessions_df = __sessions_df[__sessions_df.action_detail.isin(action_details)]
__sessions_df.shape

(4067691, 6)

In [22]:
print len(_sessions_df.action.unique()) + len(_sessions_df.action_type.unique()) + len(_sessions_df.action_detail.unique()) + len(_sessions_df.device_type.unique())
print len(__sessions_df.action.unique()) + len(__sessions_df.action_type.unique()) + len(__sessions_df.action_detail.unique()) + len(__sessions_df.device_type.unique())

485
74


In [23]:
__sessions_df.reset_index(drop=True,inplace=True)
__sessions_df.head(1)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,ey7zd06giy,search_results,click,view_search_results,Mac Desktop,5770.0


In [24]:
from sklearn.feature_extraction import DictVectorizer
 
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df

In [25]:
sessions_df = encode_onehot(__sessions_df,['action','action_type','action_detail','device_type'])
print sessions_df.shape
print 'sessions data set with dummies:'
sessions_df.head(2)

C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: the 'outtype' keyword is deprecated, use 'orient' instead


(4067691, 76)
sessions data set with dummies:


,user_id,secs_elapsed,action=12,action=NAN,action=active,action=ajax_get_referrals_amt,action=ajax_image_upload,action=ajax_lwlb_contact,action=ajax_refresh_subtotal,action=authenticate,...,device_type=Chromebook,device_type=Linux Desktop,device_type=Mac Desktop,device_type=Opera Phone,device_type=Tablet,device_type=Windows Desktop,device_type=Windows Phone,device_type=iPad Tablet,device_type=iPhone,device_type=iPodtouch
0,ey7zd06giy,5770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ey7zd06giy,496.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
act = sessions_df.columns[2:-14]
X = sessions_df[act]

In [2]:
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler

In [33]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [34]:
np.save('./x_scaled.npy',X_scaled)

In [31]:
type(X_scaled)

numpy.ndarray

In [3]:
X_scaled = np.load('./x_scaled.npy')

In [4]:
pca = PCA()


In [5]:
# pca_data = pca.fit_transform(X_scaled)

In [11]:
ratio = pca.explained_variance_ratio_
ratio[:10]

array([ 0.05916112,  0.05686092,  0.05231066,  0.0507281 ,  0.04808652,
        0.04337903,  0.03680158,  0.03509411,  0.03386958,  0.03382426])

* ______________________________________________________________ **

* How many actions (actions, action types and action details) is involved per user? 
* What are the top actions (actions, action types and action details) among users? Does it have any value for our calculations?
* What are the most time consuming actions (actions and action types, action details)

* How much time does a user spend on each session? (mean, std, sum)

In [ ]:
_2014_sessions_df.groupby(['user_id']).size().sort_values(ascending=False)

In [ ]:
id_stat =_2014_sessions_df.groupby(['user_id'])['secs_elapsed'].agg(['sum','mean','std', 'count'])

In [ ]:
id_stat.sort_values('count',ascending=False)

In [ ]:
action_details_df = pd.pivot_table(nonNDF_sessions_df_, values=['secs_elapsed'],\
                                          index=['action', 'action_detail'],aggfunc=[sum, np.mean,len],margins=True)
print action_details_df.shape
action_details_df.head()

In [ ]:
Tit_pivot_age_sex_effect = pd.pivot_table(Titanic_df, values=['survived'],columns=['sex'],\
                                          index=['age group'],aggfunc=[sum,len],margins=True)
Tit_pivot_age_sex_effect['Female Survival Ratio %'] = ((Tit_pivot_age_sex_effect['sum'])\
                                                     ['survived'])['female']*100/((Tit_pivot_age_sex_effect['len'])\
                                                                                  ['survived'])['female']
Tit_pivot_age_sex_effect['Male Survival Ratio %'] = ((Tit_pivot_age_sex_effect['sum'])\
                                                     ['survived'])['male']*100/((Tit_pivot_age_sex_effect['len'])\
                                                                                  ['survived'])['male']
Tit_pivot_age_sex_effect['Age Group Survival Ratio %'] = ((Tit_pivot_age_sex_effect['sum'])\
                                                     ['survived'])['All']*100/((Tit_pivot_age_sex_effect['len'])\
                                                                                  ['survived'])['All']
Tit_pivot_age_sex_effect